In [ ]:
# https://github.com/Element84/earth-search?tab=readme-ov-file#sns-notifications-of-items


# https://github.com/cirrus-geo/cirrus-geo

# https://cirrus-geo.github.io/cirrus-geo/v1.0.2/index.html
# https://cirrus-geo.github.io/cirrus-geo/stable/cirrus/20_arch.html

In [1]:
import json
import boto3

In [2]:
EARTH_SEARCH_TOPIC_ARN = "arn:aws:sns:us-west-2:608149789419:cirrus-es-prod-publish"
# REGION = "us-west-2"
# REGION = "us-east-1"

In [3]:
QUEUE_NAME = "IngestEarthSearch"
DLQ_NAME = "IngestEarthSearchDLQ"

## Create Services

In [4]:
FILTER_POLICY = {
    "collection": ["sentinel-2-c1-l2a"], # "landsat-c2-l2"
    # "bbox.ll_lon": {"numeric": [">=", 51.9]},
    # "bbox.ll_lat": {"numeric": [">=", 16.6]},
    # "bbox.ur_lon": {"numeric": ["<=", 59.9]},
    # "bbox.ur_lat": {"numeric": ["<=", 26.4]},
}

In [11]:
client_sqs = boto3.client(service_name="sqs", region_name="us-east-1")
client_sns = boto3.client(service_name="sns", region_name="us-west-2")

In [ ]:
# 1) Criar DLQ
# dlq_resp = client_sqs.create_queue(QueueName=DLQ_NAME)
# dlq_url = dlq_resp["QueueUrl"]

# Obter ARN da DLQ
# dlq_attrs = client_sqs.get_queue_attributes(
#     QueueUrl=dlq_url,
#     AttributeNames=["QueueArn"]
# )
# dlq_arn = dlq_attrs["Attributes"]["QueueArn"]

# print("DLQ URL:", dlq_url)
# print("DLQ ARN:", dlq_arn)

In [13]:
# 2) Criar fila principal com VisibilityTimeout (e depois setar RedrivePolicy)
q_resp = client_sqs.create_queue(
    QueueName=QUEUE_NAME,
    Attributes={
        "VisibilityTimeout": "3600",  # 3600s = 1h
    },
)
q_url = q_resp["QueueUrl"]

# Obter ARN da fila principal
q_attrs = client_sqs.get_queue_attributes(
    QueueUrl=q_url,
    AttributeNames=["QueueArn"]
)
q_arn = q_attrs["Attributes"]["QueueArn"]


# # 3) Configurar RedrivePolicy (liga a fila principal à DLQ)
# redrive_policy = {
#     "deadLetterTargetArn": dlq_arn,
#     "maxReceiveCount": "5",
# }
# client_sqs.set_queue_attributes(
#     QueueUrl=q_url,
#     Attributes={
#         "RedrivePolicy": json.dumps(redrive_policy)
#     },
# )


# 4) Aplicar QueuePolicy permitindo que o SNS (apenas esse TopicArn) envie para a fila
queue_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Allow-SNS-SendMessage",
            "Effect": "Allow",
            "Principal": {"Service": "sns.amazonaws.com"},
            "Action": "sqs:SendMessage",
            "Resource": q_arn,
            "Condition": {"ArnEquals": {"aws:SourceArn": EARTH_SEARCH_TOPIC_ARN}},
        }
    ],
}
response = client_sqs.set_queue_attributes(
    QueueUrl=q_url,
    Attributes={"Policy": json.dumps(queue_policy)},
)

print("Queue URL:", q_url)
print("Queue ARN:", q_arn)

Queue URL: https://sqs.us-east-1.amazonaws.com/891377318910/IngestEarthSearch
Queue ARN: arn:aws:sqs:us-east-1:891377318910:IngestEarthSearch


In [ ]:
# 5) Criar assinatura SNS -> SQS com FilterPolicy e FilterPolicyScope=MessageAttributes
sub_resp = client_sns.subscribe(
    TopicArn=EARTH_SEARCH_TOPIC_ARN,
    Protocol="sqs",
    Endpoint=q_arn,  # para SQS, o Endpoint é o ARN da fila
    Attributes={
        # escopo de filtragem em atributos de mensagem (igual ao CloudFormation)
        "FilterPolicyScope": "MessageAttributes",
        "FilterPolicy": json.dumps(FILTER_POLICY),
    },
    ReturnSubscriptionArn=True,
)

subscription_arn = sub_resp["SubscriptionArn"]
print("Subscription ARN:", subscription_arn)

Subscription ARN: arn:aws:sns:us-west-2:608149789419:cirrus-es-prod-publish:7085dc9d-f4f3-4a47-bcff-99dcc6fe04e6


### unsubscribe

In [ ]:
# subscription_arn = "arn:aws:sns:us-west-2:608149789419:cirrus-es-prod-publish:7085dc9d-f4f3-4a47-bcff-99dcc6fe04e6"

In [ ]:
resp = client_sns.unsubscribe(SubscriptionArn=subscription_arn)
resp

InvalidParameterException: An error occurred (InvalidParameter) when calling the Unsubscribe operation: Invalid parameter: SubscriptionId

## Receive messages

In [21]:
sqs_message = {
  "Type" : "Notification",
  "MessageId" : "73509ddf-ec7e-545f-a3bb-fc9f2a2f66f3",
  "TopicArn" : "arn:aws:sns:us-west-2:608149789419:cirrus-es-prod-publish",
  "Message" : "{\"type\": \"Feature\", \"stac_version\": \"1.0.0\", \"id\": \"S2B_T34JCL_20250914T085044_L2A\", \"properties\": {\"created\": \"2025-09-14T13:33:04.886831+00:00\", \"platform\": \"sentinel-2b\", \"constellation\": \"sentinel-2\", \"instruments\": [\"msi\"], \"eo:cloud_cover\": 1.205649, \"proj:epsg\": 32734, \"proj:centroid\": {\"lat\": -31.12884, \"lon\": 19.02775}, \"mgrs:utm_zone\": 34, \"mgrs:latitude_band\": \"J\", \"mgrs:grid_square\": \"CL\", \"grid:code\": \"MGRS-34JCL\", \"view:azimuth\": 289.92637469165317, \"view:incidence_angle\": 10.962394969236861, \"view:sun_azimuth\": 38.741420208207, \"view:sun_elevation\": 48.3035019766282, \"s2:tile_id\": \"S2B_OPER_MSI_L2A_TL_2BPS_20250914T122648_A044520_T34JCL_N05.11\", \"s2:degraded_msi_data_percentage\": 0.0, \"s2:nodata_pixel_percentage\": 79.828215, \"s2:saturated_defective_pixel_percentage\": 0.0, \"s2:cloud_shadow_percentage\": 0.000609, \"s2:vegetation_percentage\": 6.101094, \"s2:not_vegetated_percentage\": 87.931943, \"s2:water_percentage\": 0.108573, \"s2:unclassified_percentage\": 0.045725, \"s2:medium_proba_clouds_percentage\": 0.0, \"s2:high_proba_clouds_percentage\": 1.6e-05, \"s2:thin_cirrus_percentage\": 1.205633, \"s2:snow_ice_percentage\": 0.0, \"s2:product_type\": \"S2MSI2A\", \"s2:processing_baseline\": \"05.11\", \"s2:product_uri\": \"S2B_MSIL2A_20250914T082559_N0511_R021_T34JCL_20250914T122648.SAFE\", \"s2:generation_time\": \"2025-09-14T12:26:48.000000Z\", \"s2:datatake_id\": \"GS2B_20250914T082559_044520_N05.11\", \"s2:datatake_type\": \"INS-NOBS\", \"s2:datastrip_id\": \"S2B_OPER_MSI_L2A_DS_2BPS_20250914T122648_S20250914T085044_N05.11\", \"s2:reflectance_conversion_factor\": 0.986847636340467, \"datetime\": \"2025-09-14T08:58:59.657000Z\", \"earthsearch:payload_id\": \"roda-sentinel-2-c1-l2a/workflow-sentinel-2-c1-l2a-to-stac/5050f2dedaa70765a2f54d1ed4aaa097\", \"storage:platform\": \"AWS\", \"storage:region\": \"us-west-2\", \"storage:requester_pays\": false, \"processing:software\": {\"sentinel-2-c1-l2a-to-stac\": \"v2025.06.17\"}, \"updated\": \"2025-09-14T13:33:04.886831+00:00\"}, \"geometry\": {\"type\": \"Polygon\", \"coordinates\": [[[18.911418536734743, -30.715731792365737], [18.889556098630493, -31.704862067046125], [19.004592501582408, -31.70513694568981], [19.259184159284047, -30.720862701634957], [18.911418536734743, -30.715731792365737]]]}, \"links\": [{\"rel\": \"self\", \"href\": \"s3://e84-earth-search-sentinel-data/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/S2B_T34JCL_20250914T085044_L2A.json\", \"type\": \"application/json\"}, {\"rel\": \"canonical\", \"href\": \"s3://e84-earth-search-sentinel-data/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/S2B_T34JCL_20250914T085044_L2A.json\", \"type\": \"application/json\"}, {\"rel\": \"via\", \"href\": \"s3://sentinel-s2-l2a/tiles/34/J/CL/2025/9/14/0/metadata.xml\", \"type\": \"application/xml\", \"title\": \"Granule Metadata in Sinergize RODA Archive\"}], \"assets\": {\"red\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B04.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Red - 10m\", \"eo:bands\": [{\"name\": \"B04\", \"common_name\": \"red\", \"center_wavelength\": 0.665, \"full_width_half_max\": 0.038}], \"gsd\": 10, \"proj:shape\": [10980, 10980], \"proj:transform\": [10.0, 0.0, 300000.0, 0.0, -10.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 10, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"1220cca15bc470391f8321e26f174964bff2f87d6fa03cd73ea0b71485332a97d4ae\", \"file:size\": 45087687, \"roles\": [\"data\", \"reflectance\"]}, \"green\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B03.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Green - 10m\", \"eo:bands\": [{\"name\": \"B03\", \"common_name\": \"green\", \"center_wavelength\": 0.56, \"full_width_half_max\": 0.045}], \"gsd\": 10, \"proj:shape\": [10980, 10980], \"proj:transform\": [10.0, 0.0, 300000.0, 0.0, -10.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 10, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"1220d723535272c680abb9bf25ac3e4deb446ccf26f44b189b85b1125256fb0c0ee3\", \"file:size\": 43755258, \"roles\": [\"data\", \"reflectance\"]}, \"blue\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B02.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Blue - 10m\", \"eo:bands\": [{\"name\": \"B02\", \"common_name\": \"blue\", \"center_wavelength\": 0.49, \"full_width_half_max\": 0.098}], \"gsd\": 10, \"proj:shape\": [10980, 10980], \"proj:transform\": [10.0, 0.0, 300000.0, 0.0, -10.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 10, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"1220774f76c5a5e06a2c73cb53937fdd98b3516bdcf64cb455e256de5922a873dbea\", \"file:size\": 42249893, \"roles\": [\"data\", \"reflectance\"]}, \"visual\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/TCI.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"True color image\", \"eo:bands\": [{\"name\": \"B04\", \"common_name\": \"red\", \"center_wavelength\": 0.665, \"full_width_half_max\": 0.038}, {\"name\": \"B03\", \"common_name\": \"green\", \"center_wavelength\": 0.56, \"full_width_half_max\": 0.045}, {\"name\": \"B02\", \"common_name\": \"blue\", \"center_wavelength\": 0.49, \"full_width_half_max\": 0.098}], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint8\", \"spatial_resolution\": 10}, {\"nodata\": 0, \"data_type\": \"uint8\", \"spatial_resolution\": 10}, {\"nodata\": 0, \"data_type\": \"uint8\", \"spatial_resolution\": 10}], \"gsd\": 10, \"proj:shape\": [10980, 10980], \"proj:transform\": [10.0, 0.0, 300000.0, 0.0, -10.0, 6600040.0], \"file:checksum\": \"122092f822c9802a64bf7d11a0b8b18becbccb789e259c88f3b7329c5add3e64ec2f\", \"file:size\": 66007247, \"roles\": [\"visual\"]}, \"nir\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B08.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"NIR 1 - 10m\", \"eo:bands\": [{\"name\": \"B08\", \"common_name\": \"nir\", \"center_wavelength\": 0.842, \"full_width_half_max\": 0.145}], \"gsd\": 10, \"proj:shape\": [10980, 10980], \"proj:transform\": [10.0, 0.0, 300000.0, 0.0, -10.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 10, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"1220432e08d3ca008a2865446fc7dc6d71d4855fed6221a5aa8da5691106e2b7b5ef\", \"file:size\": 43378219, \"roles\": [\"data\", \"reflectance\"]}, \"swir22\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B12.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"SWIR 2.2\\u03bcm - 20m\", \"eo:bands\": [{\"name\": \"B12\", \"common_name\": \"swir22\", \"center_wavelength\": 2.19, \"full_width_half_max\": 0.242}], \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"122051dcc91d5b504214da4e0dc1008fb7d996be2bbd9a30275352749b4ca359f62c\", \"file:size\": 12037260, \"roles\": [\"data\", \"reflectance\"]}, \"rededge2\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B06.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Red Edge 2 - 20m\", \"eo:bands\": [{\"name\": \"B06\", \"common_name\": \"rededge\", \"center_wavelength\": 0.74, \"full_width_half_max\": 0.018}], \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"12202fdaf40558706ec955281d37f8b70125507a9dc6e8b9b2b37b9fb6b33ea7832e\", \"file:size\": 11951575, \"roles\": [\"data\", \"reflectance\"]}, \"rededge3\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B07.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Red Edge 3 - 20m\", \"eo:bands\": [{\"name\": \"B07\", \"common_name\": \"rededge\", \"center_wavelength\": 0.783, \"full_width_half_max\": 0.028}], \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"1220b32e60b3b3a01a83031ee0d461e50d2de2632b2b2d1871d73b71f19f83ebb0c8\", \"file:size\": 11963293, \"roles\": [\"data\", \"reflectance\"]}, \"rededge1\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B05.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Red Edge 1 - 20m\", \"eo:bands\": [{\"name\": \"B05\", \"common_name\": \"rededge\", \"center_wavelength\": 0.704, \"full_width_half_max\": 0.019}], \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"12202b325160a64d29d36fdfb56ba3cf4bd5594f7a67ed2082152221391cbcfb5308\", \"file:size\": 11964682, \"roles\": [\"data\", \"reflectance\"]}, \"swir16\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B11.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"SWIR 1.6\\u03bcm - 20m\", \"eo:bands\": [{\"name\": \"B11\", \"common_name\": \"swir16\", \"center_wavelength\": 1.61, \"full_width_half_max\": 0.143}], \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"1220b4b9e2e86f768c52ea2be345252f1048a55c50972cb2f65f3803a439e4a01a36\", \"file:size\": 12023876, \"roles\": [\"data\", \"reflectance\"]}, \"wvp\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/WVP.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Water Vapour (WVP)\", \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"unit\": \"cm\", \"scale\": 0.001, \"offset\": 0}], \"file:checksum\": \"12209d6d52d4f749e37b53ec72349d039866c30b59e8a7910ca31c0decbd5880c616\", \"file:size\": 8801047, \"roles\": [\"data\"]}, \"nir08\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B8A.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"NIR 2 - 20m\", \"eo:bands\": [{\"name\": \"B8A\", \"common_name\": \"nir08\", \"center_wavelength\": 0.865, \"full_width_half_max\": 0.033}], \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"12205ea14825664d9d4ee949f65648dbef01740d2c8f0e30e124f6d8f7d82800bf65\", \"file:size\": 11919949, \"roles\": [\"data\", \"reflectance\"]}, \"scl\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/SCL.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Scene classification map (SCL)\", \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint8\", \"spatial_resolution\": 20}], \"file:checksum\": \"1220bd2533dc5849dd5e6c01a1e597a90b712485c218a9b11bddbeb1d058d40ce250\", \"file:size\": 432147, \"roles\": [\"data\"]}, \"aot\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/AOT.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Aerosol optical thickness (AOT)\", \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 20, \"scale\": 0.001, \"offset\": 0}], \"file:checksum\": \"122074ffea9f55eb09f7e0cac0edc288b8dfc5741184bd4f220e30f74f32ed91ab16\", \"file:size\": 389482, \"roles\": [\"data\"]}, \"coastal\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B01.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Coastal - 60m\", \"eo:bands\": [{\"name\": \"B01\", \"common_name\": \"coastal\", \"center_wavelength\": 0.443, \"full_width_half_max\": 0.027}], \"gsd\": 60, \"proj:shape\": [1830, 1830], \"proj:transform\": [60.0, 0.0, 300000.0, 0.0, -60.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 60, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"1220facbf9060fb1fe04cca40f5f6623620b558375046da95b9ce88a3195eb2222b7\", \"file:size\": 1143100, \"roles\": [\"data\", \"reflectance\"]}, \"nir09\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/B09.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"NIR 3 - 60m\", \"eo:bands\": [{\"name\": \"B09\", \"common_name\": \"nir09\", \"center_wavelength\": 0.945, \"full_width_half_max\": 0.026}], \"gsd\": 60, \"proj:shape\": [1830, 1830], \"proj:transform\": [60.0, 0.0, 300000.0, 0.0, -60.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint16\", \"spatial_resolution\": 60, \"scale\": 0.0001, \"offset\": -0.1}], \"file:checksum\": \"12200b741e0dcb66645b781e9c7967481a72e94d9aae297d8cf4fedabb9bfe62b154\", \"file:size\": 1369611, \"roles\": [\"data\", \"reflectance\"]}, \"cloud\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/CLD_20m.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Cloud Probabilities\", \"gsd\": 20, \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint8\", \"spatial_resolution\": 20}], \"file:checksum\": \"12205ec301fd2009476336682a2f126b2c81ff32ce4d9ba65edfdf0b1c6691eac1d7\", \"file:size\": 67744, \"roles\": [\"data\", \"cloud\"]}, \"snow\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/SNW_20m.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"Snow Probabilities\", \"proj:shape\": [5490, 5490], \"proj:transform\": [20.0, 0.0, 300000.0, 0.0, -20.0, 6600040.0], \"raster:bands\": [{\"nodata\": 0, \"data_type\": \"uint8\", \"spatial_resolution\": 20}], \"file:checksum\": \"12204ba26f7a31aafcfdb688561f4f3c0528ffbe52c4357da8f2e39f47c09e5d64df\", \"file:size\": 54816, \"roles\": [\"data\", \"snow-ice\"]}, \"preview\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/L2A_PVI.tif\", \"type\": \"image/tiff; application=geotiff; profile=cloud-optimized\", \"title\": \"True color preview\", \"gsd\": 320, \"proj:shape\": [343, 343], \"proj:transform\": [320.0, 0.0, 300000.0, 0.0, -320.0, 6600040.0], \"eo:bands\": [{\"name\": \"B04\", \"common_name\": \"red\", \"center_wavelength\": 0.665, \"full_width_half_max\": 0.038}, {\"name\": \"B03\", \"common_name\": \"green\", \"center_wavelength\": 0.56, \"full_width_half_max\": 0.045}, {\"name\": \"B02\", \"common_name\": \"blue\", \"center_wavelength\": 0.49, \"full_width_half_max\": 0.098}], \"file:checksum\": \"1220f14b22b22c2a3022256d3dc8fd2998e213d7526073cca4b93777979ff53ab076\", \"file:size\": 70208, \"roles\": [\"overview\"]}, \"granule_metadata\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/metadata.xml\", \"type\": \"application/xml\", \"file:checksum\": \"122014cc86b0ea54b1909d87fd2eda90520946b8a294b4428f99fbf2f788eada70ea\", \"file:size\": 190046, \"roles\": [\"metadata\"]}, \"tileinfo_metadata\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/tileInfo.json\", \"type\": \"application/json\", \"file:checksum\": \"122050e878063fe86dd7cff7d5431d57f117d9ccb9a2424e0d18fd572397fd85932c\", \"file:size\": 1525, \"roles\": [\"metadata\"]}, \"product_metadata\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/product_metadata.xml\", \"type\": \"application/xml\", \"file:checksum\": \"12208e05f0e5efbe5654627f8b96e7f1359700a7388744c99027d48be3a3c8ba3a54\", \"file:size\": 55215, \"roles\": [\"metadata\"]}, \"thumbnail\": {\"href\": \"https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/34/J/CL/2025/9/S2B_T34JCL_20250914T085044_L2A/L2A_PVI.jpg\", \"type\": \"image/jpeg\", \"title\": \"Thumbnail of preview image\", \"file:checksum\": \"12209ed9fbde12310527221ab81d24c6ccec5cc3fdd39af81ab0b6273ccd8f891677\", \"file:size\": 10201, \"roles\": [\"thumbnail\"]}}, \"bbox\": [18.889556, -31.705137, 19.259184, -30.715732], \"stac_extensions\": [\"https://stac-extensions.github.io/eo/v1.1.0/schema.json\", \"https://stac-extensions.github.io/file/v2.1.0/schema.json\", \"https://stac-extensions.github.io/grid/v1.1.0/schema.json\", \"https://stac-extensions.github.io/mgrs/v1.0.0/schema.json\", \"https://stac-extensions.github.io/processing/v1.1.0/schema.json\", \"https://stac-extensions.github.io/projection/v1.1.0/schema.json\", \"https://stac-extensions.github.io/raster/v1.1.0/schema.json\", \"https://stac-extensions.github.io/sentinel-2/v1.0.0/schema.json\", \"https://stac-extensions.github.io/storage/v1.0.0/schema.json\", \"https://stac-extensions.github.io/view/v1.0.0/schema.json\"], \"collection\": \"sentinel-2-c1-l2a\"}",
  "Timestamp" : "2025-09-14T13:33:05.528Z",
  "SignatureVersion" : "1",
  "Signature" : "SL0c7PQsDG/Ud2ioJGKNB4XXVrTB0hS8gCe0QZrK+6ibb5SrGgFv2yy3Ve5liZX1ACR7UZ0WqTSVAR/aC1aREH7FFZpFwTRNPucoK92y3eFEmXJ8aVHvjL5K/4G97utP1HGuKinjmcQZWQf1oyz7Ylf8EWlZ6AtF6/uYs+coI0Z8QtCNXOR00V1hfHe+VpXOzwflnDNgo2Iw7esiiuZa2kzI4cvrgoC5PRqWdKwKNE6yhyzh5bysCSN8/VlKpmB6K7l5RWkcgn/FEYaPoKHvNoEXcDJVr3fPnOJuSX0iUSMU5mOydsr7O1y8stsdTSZLdHTQ+FoWd8ejDkkHv0okDQ==",
  "SigningCertURL" : "https://sns.us-west-2.amazonaws.com/SimpleNotificationService-6209c161c6221fdf56ec1eb5c821d112.pem",
  "UnsubscribeURL" : "https://sns.us-west-2.amazonaws.com/?Action=Unsubscribe&SubscriptionArn=arn:aws:sns:us-west-2:608149789419:cirrus-es-prod-publish:7085dc9d-f4f3-4a47-bcff-99dcc6fe04e6",
  "MessageAttributes" : {
    "datetime" : {"Type":"String","Value":"2025-09-14T08:58:59.657000Z"},
    "bbox.ur_lon" : {"Type":"Number","Value":"19.259184"},
    "start_datetime" : {"Type":"String","Value":"2025-09-14T08:58:59.657000Z"},
    "end_datetime" : {"Type":"String","Value":"2025-09-14T08:58:59.657000Z"},
    "cloud_cover" : {"Type":"Number","Value":"1.205649"},
    "bbox.ur_lat" : {"Type":"Number","Value":"-30.715732"},
    "bbox.ll_lon" : {"Type":"Number","Value":"18.889556"},
    "collection" : {"Type":"String","Value":"sentinel-2-c1-l2a"},
    "bbox.ll_lat" : {"Type":"Number","Value":"-31.705137"},
    "status" : {"Type":"String","Value":"created"}
  }
}

In [22]:
sqs_message.keys()

dict_keys(['Type', 'MessageId', 'TopicArn', 'Message', 'Timestamp', 'SignatureVersion', 'Signature', 'SigningCertURL', 'UnsubscribeURL', 'MessageAttributes'])

In [23]:
message = json.loads(sqs_message['Message'])

In [24]:
message.keys()

dict_keys(['type', 'stac_version', 'id', 'properties', 'geometry', 'links', 'assets', 'bbox', 'stac_extensions', 'collection'])

In [ ]:
with open("../ingestao_infra/sns/payload/body01.json", "w") as f:
    f.write(json.dumps(message))

In [25]:
message

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'S2B_T34JCL_20250914T085044_L2A',
 'properties': {'created': '2025-09-14T13:33:04.886831+00:00',
  'platform': 'sentinel-2b',
  'constellation': 'sentinel-2',
  'instruments': ['msi'],
  'eo:cloud_cover': 1.205649,
  'proj:epsg': 32734,
  'proj:centroid': {'lat': -31.12884, 'lon': 19.02775},
  'mgrs:utm_zone': 34,
  'mgrs:latitude_band': 'J',
  'mgrs:grid_square': 'CL',
  'grid:code': 'MGRS-34JCL',
  'view:azimuth': 289.92637469165317,
  'view:incidence_angle': 10.962394969236861,
  'view:sun_azimuth': 38.741420208207,
  'view:sun_elevation': 48.3035019766282,
  's2:tile_id': 'S2B_OPER_MSI_L2A_TL_2BPS_20250914T122648_A044520_T34JCL_N05.11',
  's2:degraded_msi_data_percentage': 0.0,
  's2:nodata_pixel_percentage': 79.828215,
  's2:saturated_defective_pixel_percentage': 0.0,
  's2:cloud_shadow_percentage': 0.000609,
  's2:vegetation_percentage': 6.101094,
  's2:not_vegetated_percentage': 87.931943,
  's2:water_percentage': 0.108573,
  